In [11]:
import numpy as np
import pandas as pd
import math

from id3 import ID3DecisionTree

In [14]:
unique_arr = [0, 0, 0, 0, 1]
np.unique(unique_arr, return_counts=True)

(array([0, 1]), array([4, 1], dtype=int64))

# Classes

# Helper Functions

In [2]:
def matrix_of_attrs_belonging_to_labels(attr_label_arr):
    """Computes matrix matching p_i, n_i, ... for any number of values.

    The notation p_i and n_i is from 
    https://hunch.net/~coms-4771/quinlan.pdf

    :param attr_label_arr: 2D <class 'numpy.ndarray'> where the first column
        is the attribute and the values that the attribute can take 
        on while the second column is the labels (classification)
        for each value of the attribute.

    :return: <class 'list'> of <class 'list'>
    """
    
    # Compute the unique discrete values that the attributes can
    # take on
    unique_attr_values = np.unique(attr_label_arr[:, 0])

    # Compute the unique discrete values that the labels can
    # take on
    unique_label_values = np.unique(attr_label_arr[:, 1])

    # A matrix with 1 row for each discrete value the attribute
    # can take on and 1 column for each discrete value the 
    # label can take on. This data structure holds the counts
    # of each attribute value that matches the discrete label value
    # Example, 
    # ['overcast' 'rain' 'sunny'] for labels = {0, 1}
    # outlook [[0, 4], [2, 3], [3, 2]]
    # This means that the outlook attribute which can take on 3 discrete
    # values has 0 overcast days that are also labeled 0, while it has 
    # 4 overcast days that are also labeled 1.
    attr_label_count_matrix = []

    # Iterate through each unique value of the attributes
    # and count the occurences for which the attribute value 
    # is unique value AND the label is a unique value of that label
    for unique_attr_value in unique_attr_values:
        counts = []
        for unique_label_value in unique_label_values:
            # Boolean vector... elements are True where condition
            # holds, False otherwise
            match_vector = np.logical_and(
                attr_label_arr[:, 0] == unique_attr_value, 
                attr_label_arr[:, 1] == unique_label_value)

            # Non-zero means the instances in which the condition is True
            match_count = np.count_nonzero(match_vector)

            # Append the value to the counts list
            counts.append(match_count)

        # Append the counts list to the parent matrix
        attr_label_count_matrix.append(counts)

    # Resulting matrix
    return attr_label_count_matrix

def discretize_continuous_data(attr_label_arr, return_bins=False):
    """Converts continuous attributes to discrete valued array."""
    
    threshold_indices = get_threshold_indices(attr_label_arr)
    discretized_arr, bins = get_binned_arrs(attr_label_arr, threshold_indices, return_bins)

    if return_bins:
        return discretized_arr, bins
    else:
        return discretized_arr

def get_threshold_indices(attr_label_arr):
    """Returns list of indices where adjacent labels differ.
    
    Array should be sorted.
    """

    # Check sorted
    if attr_label_arr[0, 0] > attr_label_arr[1, 0]:
      raise ValueError(':param attr_label_arr: must be sorted')

    threshold_tup_indices = []

    for row in range(len(attr_label_arr)-1):
        cur_label = attr_label_arr[row, -1]
        next_label = attr_label_arr[row+1, -1]
        
        if cur_label != next_label:
            threshold_tup_indices.append((row, row+1))

    return threshold_tup_indices

def get_binned_arrs(attr_label_arr, threshold_indices, return_bins):
    """Use thresholds to bin the sorted data."""

    # Compute the bins
    bins = []
    for threshold_ix in threshold_indices:
        bin_ = (attr_label_arr[threshold_ix[0], 0] \
            + attr_label_arr[threshold_ix[1], 0]) / 2
        bins.append(bin_)

    # Discretize the data into binned arrs
    lst_of_discretized_arrs = []
    for b in bins:
        # Binary thresholds
        discretized_attr_label_arr = attr_label_arr.copy()
        for ix, row in enumerate(outlook_sunny_arr):

            # attr > bin_ or can be framed as attr >= bin_
            if row[0] > b:
                discretized_attr_label_arr[ix, 0] = 0
            else:
                discretized_attr_label_arr[ix, 0] = 1

        lst_of_discretized_arrs.append(discretized_attr_label_arr)

    if return_bins:
        return lst_of_discretized_arrs, bins
    else:
        return lst_of_discretized_arrs

def sort_arr(attr_label_arr):
    """Sorts the array and keeps labels lined up."""

    sorted_attr_indices = np.argsort(attr_label_arr[:, 0])
    attr_label_arr = attr_label_arr[sorted_attr_indices, :]
    return attr_label_arr

# Instantiate Tree

In [3]:
# Instantiate tree
tree = ID3DecisionTree(X=None, y=None)

# Load Example Data

In [4]:
df = pd.read_excel('sample_data.xlsx')

In [5]:
df

,outlook,temperature,humidity,windy,class
0,sunny,hot,high,False,N
1,sunny,hot,high,True,N
2,overcast,hot,high,False,P
3,rain,mild,high,False,P
4,rain,cool,normal,False,P
5,rain,cool,normal,True,N
6,overcast,cool,normal,True,P
7,sunny,mild,high,False,N
8,sunny,cool,normal,False,P
9,rain,mild,normal,False,P


In [6]:
# Testing helper function
outlook_class_arr = df[['outlook', 'class']].to_numpy()
display(outlook_class_arr)

matrix_of_attrs_belonging_to_labels(outlook_class_arr)

array([['sunny', 'N'],
       ['sunny', 'N'],
       ['overcast', 'P'],
       ['rain', 'P'],
       ['rain', 'P'],
       ['rain', 'N'],
       ['overcast', 'P'],
       ['sunny', 'N'],
       ['sunny', 'P'],
       ['rain', 'P'],
       ['sunny', 'P'],
       ['overcast', 'P'],
       ['overcast', 'P'],
       ['rain', 'N']], dtype=object)

[[0, 4], [2, 3], [3, 2]]

In [7]:
tree.entropy([3, 2])

0.9709505944546686

In [8]:
tree.expected_information(label_counts=[9, 5], attr_counts=[[0, 4], [2, 3], [3, 2]])

0.6935361388961918

In [9]:
# information gain for outlook
tree.information_gain(tree.entropy([9, 5]), 
    tree.expected_information(
        label_counts=[9, 5], attr_counts=[[0, 4], [2, 3], [3, 2]]))

0.2467498197744391

# Algorithm
1. E(classification)
2. For each feature <br/>
2.1. Discretize if not already discrete <br/>
2.2. Calculate Expected Information Gain w.r.t classification <br/>
2.3. Calculate information gain for the feature and store in list <br/>
3. argmax[lst_of_feature_info_gains]
4. Repeat
    

# Full Algorithm on Only Categorical Dataset
https://iq.opengenus.org/id3-algorithm/

In [10]:
# Convert df to arr
arr = df.to_numpy()

labels_vector = arr[:, -1]

# Entropy of whole set
_, unique_label_counts = np.unique(labels_vector, return_counts=True)
entropy_s = tree.entropy(unique_label_counts)

# List to hold information gain
information_gain_lst = []

# Iterate through features (last ix is label)
for feature in range(df.shape[1] - 1):

    # Compute the number of discrete values of a given feature
    # that belong to each of the discrete values of the label
    attr_label_count_matrix = tree.matrix_of_attrs_belonging_to_labels(arr[:, [feature, -1]])

    # Compute the expected information
    expected_information = tree.expected_information(unique_label_counts, attr_label_count_matrix)

    # Compute the information gain
    information_gain = tree.information_gain(entropy_s, expected_information)

    # Append to the list of information gains
    information_gain_lst.append(information_gain)

    # Log information gains
    print(df.columns[feature], ':', information_gain)

# Select attribute with maximum information gain

# Check to see if pure class

# Call the function again except now with regard to one of 
# values that the selected attribute can take on


outlook : 0.2467498197744391
temperature : 0.029222565658954647
humidity : 0.15183550136234136
windy : 0.04812703040826927


## Data is First Sorted

In [50]:
# 3.3. Dealing with Continuous Valued Data
outlook_sunny_arr = np.array(
    [[0.9, 0], 
    [0.72, 1], 
    [0.87, 0], 
    [0.68, 1], 
    [0.91, 0]])
    # [0.93, 2]]) # optional for observing multiple thresholds

# Print before sorting
print(outlook_sunny_arr.shape)
display(outlook_sunny_arr)

# Sort data
outlook_sunny_arr = sort_arr(outlook_sunny_arr)

# After sorting
display(outlook_sunny_arr)

(5, 2)


array([[0.9 , 0.  ],
       [0.72, 1.  ],
       [0.87, 0.  ],
       [0.68, 1.  ],
       [0.91, 0.  ]])

array([[0.68, 1.  ],
       [0.72, 1.  ],
       [0.87, 0.  ],
       [0.9 , 0.  ],
       [0.91, 0.  ]])

## Discretize data (assume continuous?)

In [51]:
# Discretize the above data based on the paper
discrete_outlook_sunny_data, bins = discretize_continuous_data(outlook_sunny_arr, True)
display(bins)
display(discrete_outlook_sunny_data)

[0.7949999999999999]

[array([[1., 1.],
        [1., 1.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])]

## Expected information gain

In [52]:
for discrete_arr in discrete_outlook_sunny_data:
    attr_count_matrix_outlook_sunny_data = matrix_of_attrs_belonging_to_labels(discrete_arr)
    print('The 0 discrete attr has 3 instances that belong to the 0 label')
    display(attr_count_matrix_outlook_sunny_data)

The 0 discrete attr has 3 instances that belong to the 0 label


[[3, 0], [0, 2]]

In [53]:
# Cleaner code
discrete_count_lst = [
    matrix_of_attrs_belonging_to_labels(discrete_arr) 
    for discrete_arr in discrete_outlook_sunny_data]

discrete_count_lst

[[[3, 0], [0, 2]]]

## Calculate whichever split pattern yields the best information gain

In [54]:
# Is label counts wrt to the whole set or just the subset???
# based on "implementation of ID3", it is respect
# to subset
_, label_counts_for_subset = np.unique(discrete_arr[:, 1], return_counts=True)
entropy_of_subset = tree.entropy(label_counts_for_subset)
print('label counts for subset:', label_counts_for_subset)
print('entropy of subset:', entropy_of_subset)

expected_information_lst = [
    tree.expected_information(label_counts_for_subset, attr_counts) 
    for attr_counts in discrete_count_lst]

print('expected information gain based on splitting:', expected_information_lst)

print('information gain', [
    tree.information_gain(entropy_of_subset, expect_info_gain) 
    for expect_info_gain in expected_information_lst])

label counts for subset: [3 2]
entropy of subset: 0.9709505944546686
expected information gain based on splitting: [0.0]
information gain [0.9709505944546686]


# Misc

In [20]:
unsorted_outlook_sunny_arr = np.array(
    [[0.9, 0], 
    [0.72, 1], 
    [0.87, 0], 
    [0.68, 1], 
    [0.91, 0]]) 

sorted_rows = np.argsort(unsorted_outlook_sunny_arr[:, 0], )
unsorted_outlook_sunny_arr[sorted_rows, :]

array([[0.68, 1.  ],
       [0.72, 1.  ],
       [0.87, 0.  ],
       [0.9 , 0.  ],
       [0.91, 0.  ]])

In [34]:
# To pick thresholds, find all indices for which 
# the class (contained in the left most column) differ between
# adjacent rows
threshold_tup_indices = []
for row in range(len(outlook_sunny_arr)-1):
    cur_label = outlook_sunny_arr[row, -1]
    next_label = outlook_sunny_arr[row+1, -1]
    if cur_label != next_label:
        threshold_tup_indices.append((row, row+1))

print(threshold_tup_indices)
for threshold_ix in threshold_tup_indices:
    bin_ = (outlook_sunny_arr[threshold_ix[0], 0] \
        + outlook_sunny_arr[threshold_ix[1], 0]) / 2

    print(bin_)
    print(outlook_sunny_arr[threshold_ix[0]: threshold_ix[1]+1])
    print()

[(1, 2)]
0.7949999999999999
[[0.72 1.  ]
 [0.87 0.  ]]



In [38]:
# Bin data -- > and <=, should be reversed for open lab
bin_ = 0.795
discretized_outlook_sunny_arr = outlook_sunny_arr.copy()
for ix, row in enumerate(outlook_sunny_arr):
    if row[0] > bin_:
        discretized_outlook_sunny_arr[ix, 0] = 0
    else:
        discretized_outlook_sunny_arr[ix, 0] = 1

print(discretized_outlook_sunny_arr)

[[1. 1.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [ ]:
unique_class_labels = np.unique(df['class'])
print(unique_class_labels)
for col in df.columns[:-1]:
    col_arr = df[[col, 'class']].to_numpy()
    unique_col_values = np.unique(df[col])
    print(unique_col_values)
    n_i = []
    for unique_val in unique_col_values:
        pairs = []
        for unique_label in unique_class_labels:
            pairs.append(
                np.count_nonzero(
                    np.logical_and(col_arr[:, 0] == unique_val, col_arr[:, 1] == unique_label)))
        n_i.append(pairs)

    print(col, n_i)

['N' 'P']
['overcast' 'rain' 'sunny']
outlook [[0, 4], [2, 3], [3, 2]]
['cool' 'hot' 'mild']
temperature [[1, 3], [2, 2], [2, 4]]
['high' 'normal']
humidity [[4, 3], [1, 6]]
[False  True]
windy [[2, 6], [3, 3]]


In [ ]:
simple_arr = arr[:, [0, -1]]
print(simple_arr)
np.count_nonzero(np.logical_and(simple_arr[:, 0] == 'overcast', 
    simple_arr[:, 1] == 1))

[['sunny' 1]
 ['sunny' 1]
 ['sunny' 1]
 ['sunny' 1]
 ['sunny' 1]
 ['overcast' 1]
 ['overcast' 1]
 ['overcast' 1]
 ['overcast' 1]
 ['rain' 0]
 ['rain' 0]
 ['rain' 0]
 ['rain' 0]
 ['rain' 0]]


4

In [ ]:
tree.information(labels_counts)

0.9402859586706309